# Preprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

from google.colab import files
uploaded = files.upload()


#  Import and read the charity_data.csv.
#import pandas as pd 
#application_df = pd.read_csv("charity_data.csv")
#application_df.head()

Saving charity_data.csv to charity_data.csv


In [2]:
#  Import and read the charity_data.csv.

import pandas as pd

import io
 

application_df = pd.read_csv(io.BytesIO(uploaded['charity_data.csv']))

print(application_df)

             EIN                                               NAME  \
0       10520599                       BLUE KNIGHTS MOTORCYCLE CLUB   
1       10531628             AMERICAN CHESAPEAKE CLUB CHARITABLE TR   
2       10547893                 ST CLOUD PROFESSIONAL FIREFIGHTERS   
3       10553066                     SOUTHSIDE ATHLETIC ASSOCIATION   
4       10556103           GENETIC RESEARCH INSTITUTE OF THE DESERT   
...          ...                                                ...   
34294  996009318              THE LIONS CLUB OF HONOLULU KAMEHAMEHA   
34295  996010315           INTERNATIONAL ASSOCIATION OF LIONS CLUBS   
34296  996012607                                PTA HAWAII CONGRESS   
34297  996015768  AMERICAN FEDERATION OF GOVERNMENT EMPLOYEES LO...   
34298  996086871                           WATERHOUSE CHARITABLE TR   

      APPLICATION_TYPE       AFFILIATION CLASSIFICATION      USE_CASE  \
0                  T10       Independent          C1000    ProductDev   
1

In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(["EIN", "NAME"], axis = 1)
# application_df = application_df.drop('NAME', axis=1)
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [5]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [6]:
# Look at APPLICATION_TYPE value counts for binning
appCnt = application_df['APPLICATION_TYPE'].value_counts()
appCnt

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(appCnt[appCnt<500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Look at CLASSIFICATION value counts for binning
valCntBin = application_df['CLASSIFICATION'].value_counts()

In [10]:
# You may find it helpful to look at CLASSIFICATION value counts >1
cntClass = valCntBin[valCntBin>1]
cntClass

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [11]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list (valCntBin[valCntBin <100].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64

In [12]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df,dtype=float)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [13]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL'].values
X = application_df.drop('IS_SUCCESSFUL', axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 42)

In [14]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Compile, Train, and Evaluate

In [15]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
inputFeatures = len(X_train_scaled[0])
hiddenNodes1 = 7
hiddenNodes2 = 14

# 7 14
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hiddenNodes1, input_dim = inputFeatures, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hiddenNodes2,  activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 7)                 350       
                                                                 
 dense_1 (Dense)             (None, 14)                112       
                                                                 
 dense_2 (Dense)             (None, 1)                 15        
                                                                 
Total params: 477
Trainable params: 477
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Compile the model
nn.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])

In [17]:
# Train the model
fitModel = nn.fit(X_train_scaled,y_train,validation_split=0.15, epochs=100)

Epoch 1/100
684/684 [==============================] - 4s 4ms/step - loss: 0.6181 - accuracy: 0.6938 - val_loss: 0.5613 - val_accuracy: 0.7349
Epoch 2/100
684/684 [==============================] - 2s 3ms/step - loss: 0.5667 - accuracy: 0.7219 - val_loss: 0.5499 - val_accuracy: 0.7375
Epoch 3/100
684/684 [==============================] - 2s 3ms/step - loss: 0.5610 - accuracy: 0.7251 - val_loss: 0.5486 - val_accuracy: 0.7344
Epoch 4/100
684/684 [==============================] - 2s 2ms/step - loss: 0.5579 - accuracy: 0.7245 - val_loss: 0.5454 - val_accuracy: 0.7380
Epoch 5/100
684/684 [==============================] - 2s 2ms/step - loss: 0.5559 - accuracy: 0.7258 - val_loss: 0.5442 - val_accuracy: 0.7401
Epoch 6/100
684/684 [==============================] - 3s 4ms/step - loss: 0.5542 - accuracy: 0.7263 - val_loss: 0.5452 - val_accuracy: 0.7385
Epoch 7/100
684/684 [==============================] - 3s 4ms/step - loss: 0.5532 - accuracy: 0.7282 - val_loss: 0.5436 - val_accuracy: 0.7365

In [18]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5538 - accuracy: 0.7300 - 427ms/epoch - 2ms/step
Loss: 0.5537973642349243, Accuracy: 0.7300291657447815


In [19]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_Optimization.h5')